In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 22 14:13:15 2023

@author: souchaud
"""
import math
import os
import sys
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import trackpy as tp
import functions_analyze as lib
from scipy.signal import find_peaks

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device for torch operations:", device)

In [ ]:
def force_field_inbox(coordinates_diff, distances, Req, R0, Frep, Fadh,
                      coeff_a = None, coeff_rep = None):
    """
    Calculate the force field within the focal box.

    Parameters
    ----------
    - coordinates_diff: Tensor representing the positions of particles in the focal box.

    Returns
    -------
    - force_field: Tensor representing the force field within the focal box.

    """  
    Rlim = 0.000001
    R = torch.norm(coordinates_diff, dim=2)
    # prevents the repulsion force from exploding when approaching its maximum value
    R = torch.where(R > Rlim*torch.ones(1, device=device),
                  R, Rlim*torch.ones(1, device=device)).to(device)
    
    ###########  R**2 adhesion force ###########
    # a = coeff_a
    # b = (Fadh-a*(R0**2-Req**2))/(R0-Req)
    # c = -Req*(a*Req+ (Fadh-a*(R0**2-Req**2))/(R0-Req))
    # force = torch.where(torch.logical_and(R < R0, R > Req),
    #                     -(a*R**2+b*R+c), torch.zeros_like(R)).to(device)
    
    ########### a*R**alpha + b ############
    alpha = coeff_a
    # force = torch.where(torch.logical_and(R < R0, R > Req),
    #                     -(Fadh/((R0**alpha)-(Req**alpha)))*((R**alpha)-(Req**alpha)), torch.zeros_like(R)).to(device)
    force = torch.where(torch.logical_and(R < R0, R > Req),
                        function_adh(R, Req, R0, Fadh, alpha, coeff_a=coeff_a), torch.zeros_like(R)).to(device)

    # The repulsion force is calculated wherever R<Req
    ########### Linear adhesion force ########### 
    # force = torch.where(torch.logical_and(R < R0, R > Req),
    #                     -((Fadh/(R0-Req))*R-Fadh*Req/(R0-Req)), torch.zeros_like(R)).to(device)

    ###########  Repulsion force linear ########## 
    # force = torch.where(R < Req,
    #                     - Frep*R*(1/Req-1/R), force).to(device)
    ########### Repulsion forces in 1/R ########### 
    force = torch.where(R <= Req,
                        - Frep*coeff_rep*(1/Req-1/R), force).to(device)

    force_field = torch.sum(force[:, :, None] *
                            torch.nn.functional.normalize(coordinates_diff, dim = 2), axis=1)

    return force_field


def function_adh(R, Req, R0, Fadh, alpha, coeff_a):
    # a = coeff_a
    # b = (Fadh-a*(R0**2-Req**2))/(R0-Req)
    # c = -Req*(a*Req+ (Fadh-a*(R0**2-Req**2))/(R0-Req))
    # return -(a*R**2+b*R+c)
    return -((Fadh/(R0-Req))*R-Fadh*Req/(R0-Req))


def plot_environment(cells, space_size,req, path_saving=None, iteration=None):
    fig, axis = plt.subplots(figsize=(6, 6))
    plt.xlim(0, space_size)
    plt.ylim(0, space_size)

    # Combine cells from both populations
    all_cells = population1.cells + population2.cells

    # Extract x and y coordinates
    x = [cell.position[0].item() for cell in all_cells]
    y = [cell.position[1].item() for cell in all_cells]

    # Create a list of colors corresponding to each cell
    colors = ['blue'] * len(population1.cells) + \
        ['red'] * len(population2.cells)

    # Plot all cells at once with the specified colors
    axis.scatter(x, y, s=3, color=colors,alpha=0.5, rasterized=True)

    # plt.title('Cell Movement')
    plt.xlabel('X position (micrometers)')
    plt.ylabel('Y position (micrometers)')
    # plt.axis('off')
    plt.axis('off')
    plt.savefig(f'{path_saving}image_{iteration}.png',
                bbox_inches='tight', dpi=400, pad_inches = 0)

    # plt.show()
    plt.close()
    # print(iteration)


def plot_function(pas, Req, R0, Frep, Fadh, a, coeff_rep):

    b = (Fadh-a*(R0**2-Req**2))/(R0-Req)
    c = -Req*(a*Req + (Fadh-a*(R0**2-Req**2))/(R0-Req))

    fig, axis = plt.subplots(figsize=(6, 6))
    plt.xlim(0, R0)
    plt.ylim(-Frep, Fadh)

    print("Req = ", Req)
    print("R0 = ", R0)
    print("Fadh = ", Fadh)
    print("Frep = ", Frep)

    axis.plot(np.arange(pas, Req, pas), [
              R*Frep*(1/Req-1/R) for R in np.arange(pas, Req, pas)], label='rep Mathieu')
    axis.plot(np.arange(pas, Req, pas), [
              Frep*coeff_rep*(1/Req-1/R) for R in np.arange(pas, Req, pas)], label='rep Alex')

    axis.plot(np.arange(Req, R0, pas), [
              (Fadh/(R0-Req))*(R-Req) for R in np.arange(Req, R0, pas)], label='adhline')
    axis.plot(np.arange(Req, R0, pas), [-function_adh(R, Req, R0, Fadh, alpha=0.5, coeff_a=30)
                                        for R in np.arange(Req, R0, pas)], alpha=0.5, label='adh_Alex')
    axis.plot(np.arange(Req, R0, pas), [(a*R**2+b*R+c)
              for R in np.arange(Req, R0, pas)], label="square")

    # (Fadh/(R0-Req))*R+Fadh*Req/(R0-Req)

    plt.xlabel('Distance')
    plt.ylabel('Force')
    plt.legend()
    plt.show()

In [ ]:
class CellAgent:
    def __init__(self, id, pop, position, velocity,
                 velocity_magnitude, persistence, space_size,
                 tau, noise):
        self.id = id  # ID unique de la cellule
        self.pop = pop
        self.position_init = position.clone().to(device)
        self.position = position.clone().to(device)
        self.velocity = velocity.clone().to(device)
        self.velocity_magnitude = velocity_magnitude
        self.persistence = persistence
        self.space_size = space_size
        self.tau = tau
        self.noise = noise
        self.direction = torch.nn.functional.normalize(velocity, p=2, dim=0)

In [ ]:
class Population:
    def __init__(self, num_cells, space_size, velocity_magnitude,
                 persistence, min_distance, pop_tag, ecart_type,
                 tau, noise):
        self.num_cells = num_cells
        self.space_size = space_size
        self.velocity_magnitude = velocity_magnitude
        self.persistence = persistence
        self.min_distance = min_distance
        self.pop_tag = pop_tag  # Identifiant de la population
        self.ecart_type = ecart_type  # Initialisation de l'écart type
        self.tau = tau
        self.noise = noise
        self.cells = []
        self.initialize_cells()

    def initialize_cells(self):
        global cell_id_counter  # Utilisez cell_id_counter comme un compteur global
        positions = torch.rand((self.num_cells, 2), device=device) * self.space_size
        directions = torch.nn.functional.normalize(torch.empty_like(positions).uniform_(-1, 1), dim=1)
        vitesses_gaussiennes = torch.normal(mean=self.velocity_magnitude, std=self.ecart_type, size=(self.num_cells,))

        # Logique pour placer les cellules en respectant la distance minimale
        if self.min_distance != 0:
            grid_size = int(np.ceil(self.space_size / self.min_distance))
            grid = [[[] for _ in range(grid_size)] for _ in range(grid_size)]
            
            for i, position in enumerate(positions):
                # Logique de placement en respectant la distance minimale
                placed = False
                while not placed:
                    grid_x = int(position[0] / self.min_distance)
                    grid_y = int(position[1] / self.min_distance)
                    conflicts = any(torch.norm(position - other_position) < self.min_distance for other_position in grid[grid_x][grid_y])
                    if not conflicts:
                        for dx in [-1, 0, 1]:
                            for dy in [-1, 0, 1]:
                                if dx == 0 and dy == 0:
                                    continue
                                nx, ny = grid_x + dx, grid_y + dy
                                if 0 <= nx < grid_size and 0 <= ny < grid_size:
                                    conflicts = any(torch.norm(position - other_position) < self.min_distance for other_position in grid[nx][ny])
                                    if conflicts:
                                        break
                            if conflicts:
                                break
                    if not conflicts:
                        placed = True
                        grid[grid_x][grid_y].append(position)
                        # velocity = directions[i] * vitesses_gaussiennes[i]
                        # self.cells.append(CellAgent(cell_id_counter, position, velocity, vitesses_gaussiennes[i], self.persistence, self.space_size))
                        # cell_id_counter += 1
                        velocity = directions[i] * vitesses_gaussiennes[i]
                        self.cells.append(CellAgent(cell_id_counter, self.pop_tag, position, velocity,
                                                    vitesses_gaussiennes[i], self.persistence, self.space_size,
                                                    self.tau, self.noise,
                                                    ))
                        cell_id_counter += 1
                        placed = True  # Assurez-vous que cela est correctement géré dans votre logique
                    else:
                        position = torch.rand(2, device=device) * self.space_size
        else:
            # for i, position in enumerate(positions):
            #     velocity = directions[i] * vitesses_gaussiennes[i]
            #     self.cells.append(CellAgent(cell_id_counter, position, velocity, vitesses_gaussiennes[i], self.persistence, self.space_size))
            #     cell_id_counter += 1
            for i, position in enumerate(positions):
                velocity = directions[i] * vitesses_gaussiennes[i]
                self.cells.append(CellAgent(cell_id_counter, self.pop_tag, position, velocity,
                                                    vitesses_gaussiennes[i], self.persistence, self.space_size,
                                                    self.tau, self.noise,
                                                    ))
                cell_id_counter += 1

In [ ]:
class Surface:
    def get_friction(self, position):
        friction = torch.empty(1).uniform_(0, 0.2).to(device)
        return friction.item()

Définitions des différents paramètres de la simulation

In [ ]:
PLOT = True
# In[Simulation parameters]
# Space parameters
SPACE_SIZE = 2048 # 2048 #1308 # Micrometers

# time settings
TIME_SIMU = 90 # time simulation in minutes
DELTA_T = 0.01 # 0.01 # 15/60 # 0.01 # simulation interval in minutes
PLOT_INTERVAL = 100 # 25

# simulation parameters
MU = 1 #1  # mobility in min.kg-1
F_REP = 40  # repulsive strength
F_ADH = 7 # 3 #4 #attractive strength force kg.um.min-2
R_EQ = 1.1 # 11  # equilibrium radius in um
R_0 = 1.6 # 16  # interaction radius in um
MIN_DISTANCE_INIT = R_EQ
COEFF_CARRE = 50
COEFF_REP = 0.5
plot_function(pas=0.01, Req=R_EQ, R0=R_0, Frep=F_REP,
              Fadh=F_ADH, a=COEFF_CARRE, coeff_rep=COEFF_REP)

# factor simulation
FLUCTUATION_FACTOR = 3 #4

# Cells definition
PACKING_FRACTION = 0.00005
N_CELLS = int((PACKING_FRACTION*SPACE_SIZE**2)/
              (math.pi*((R_EQ/2)**2)))  # number of particles
print(N_CELLS, "cells")
# Population 1 parameters
velocity_magnitude_pop1 = 3 #um/min
ECART_TYPE_POP1 = 0.3
NOISE_POP_1 = 8 # noise intensity on the angle at each step
TAU_POP_1 = 5 # characteristic time for the polarization to align in the scattering direction defined by v=dr/dt = time
PERSISTENCE_POP1 = 0 #0.1
# Population 2 parameters
velocity_magnitude_pop2 = 8 # um/min
ECART_TYPE_POP2 = 0.5
NOISE_POP_2 = 5
TAU_POP_2 = 5
PERSISTENCE_POP2 = 0 #0.4

In [ ]:
cell_id_counter = 0  # Commencez le comptage des ID à partir de 0
# In[Definition of the populations]
population1 = Population(num_cells=int(N_CELLS/2), space_size=SPACE_SIZE,
                         velocity_magnitude=velocity_magnitude_pop1,
                         persistence=PERSISTENCE_POP1, ecart_type=ECART_TYPE_POP1,
                         min_distance=MIN_DISTANCE_INIT, pop_tag="Population 1",
                         tau = TAU_POP_1, noise=NOISE_POP_1)

population2 = Population(num_cells=int(N_CELLS/2), space_size=SPACE_SIZE,
                         velocity_magnitude=velocity_magnitude_pop2,
                         persistence=PERSISTENCE_POP2, ecart_type=ECART_TYPE_POP2,
                         min_distance=MIN_DISTANCE_INIT, pop_tag="Population 2",
                         tau = TAU_POP_2, noise=NOISE_POP_2)

cells = population1.cells + population2.cells

surface = Surface()

In [ ]:
# In[initialisation]
positions = torch.stack([cell.position_init for cell in cells])
V0 = torch.tensor([cell.velocity_magnitude for cell in cells], device = device).unsqueeze(1)
direction = torch.stack([cell.direction for cell in cells])
positions = torch.stack([cell.position for cell in cells])

if PLOT : 
    PATH = f'/Users/souchaud/Desktop/simu/v1{velocity_magnitude_pop1}v2{velocity_magnitude_pop2}a{COEFF_CARRE}coefrep{COEFF_REP}fadh{F_ADH}frep{F_REP}/'
    if not os.path.exists(PATH):
        os.mkdir(PATH)
    else :
        print("WARNING : FOLDER DOES ALREADY EXIST!")
        sys.exit(0)
    plot_environment(cells, space_size= SPACE_SIZE, req=R_EQ,
                    path_saving=PATH, iteration = 0)

In [ ]:
def autovel(dX, n, tau, noise, dt, persistence):
    """
    Calcule la nouvelle direction des cellules basée sur leur déplacement,
    leur direction précédente, le bruit, le temps caractéristique tau, et la persistance.
    
    Paramètres
    ----------
    dX : Tensor
        Le déplacement des cellules durant le dernier intervalle de temps.
    n : Tensor
        La direction actuelle des cellules.
    tau : float
        Le temps caractéristique pour l'alignement de la polarisation dans
        la direction définie par la vitesse = dX/dt.
    noise : float
        L'intensité du bruit ajouté pour simuler des perturbations aléatoires
        dans la direction des cellules.
    dt : float
        L'intervalle de temps entre les mises à jour des positions.
    persistence : float
        Le facteur de persistance qui influence à quel point la direction actuelle
        est conservée. Valeur entre 0 (pas de persistance, direction entièrement aléatoire)
        et 1 (persistance complète, pas de changement de direction).

    Retour
    -------
    new_direction : Tensor
        La nouvelle direction des cellules après mise à jour.
    """    
     # Normalize the input vector
    dX_norm = torch.nn.functional.normalize(dX, dim=1) * 0.9999999
    if persistence == 1:
        persistence = 0.9999999
    # Compute the angle between the input vector and the x-axis
    theta = torch.atan2(dX_norm[:, 1], dX_norm[:, 0]).to(device)
    
    # Compute the change in angle based on the orientation vector (n)
    dtheta = torch.arcsin((n[:, 0] * dX_norm[:, 1] - n[:, 1] * dX_norm[:, 0])) * dt / tau # * (1 - persistence))

    # Generate random noise for angle perturbation
    rnd = (2 * math.pi * (torch.rand(len(dX), 1, device=device) - 0.5)) * noise * np.sqrt(dt) #* (1 - persistence)
    
    # Capture de theta avant sa mise à jour
    initial_theta = theta.clone()  # Utilisez clone() pour éviter les modifications inattendues
    # Update the angle by adding the change in angle, random noise, and the previous angle
    theta_update = initial_theta + dtheta + rnd.squeeze(1)

    # Calculate the weighted average of the new direction and the old direction
    new_dir_x = torch.cos(theta_update) #* (1 - persistence) + n[:, 0] * persistence
    new_dir_y = torch.sin(theta_update) #* (1 - persistence) + n[:, 1] * persistence
    
    # Créer un nouveau tenseur pour la direction mise à jour au lieu de modifier n
    new_direction = torch.stack((new_dir_x, new_dir_y), dim=1)
    
    # Retourne la nouvelle direction, rnd, dtheta, et initial_theta
    return new_direction

In [ ]:
# Print the circle to compare angles 
        # if abs(angle) > 1.5:
        #     print("Pop1", i, "direct", cell.direction, "newdire", new_direction,
        #           "angle", angle, "disp", cell_displacement, "init theta", initial_theta,
        #           "random", rnd)
        #     fig, ax = plt.subplots(figsize=(6, 6))
        #     # Cercle unitaire
        #     circle = plt.Circle((0, 0), 1, color='blue', fill=False)
        #     ax.add_artist(circle)
        #     # Ajouter les vecteurs
        #     for direction, color, label in zip([cell.direction.unsqueeze(0), new_direction], ['red', 'green'], ['n', 'new_direction']):
        #         ax.quiver(0, 0, direction[0, 0], direction[0, 1], angles='xy', scale_units='xy', scale=1, color=color, label=label)
        #     # Configuration du graphique
        #     ax.set_xlim(-1.5, 1.5)
        #     ax.set_ylim(-1.5, 1.5)
        #     ax.set_aspect('equal')
        #     plt.xlabel('X')
        #     plt.ylabel('Y')
        #     plt.title('Vecteurs sur un cercle trigonométrique')
        #     plt.legend()
        #     plt.grid(True)
        #     plt.show()

In [ ]:
# In[Simulation]
time = 0
iteration = 1
MAX_DISTANCE = np.sqrt(2*(SPACE_SIZE/2)**2)
# Création du data_list vide pour créer la liste qui deviendra le dataFrame
data_list = []
while time < TIME_SIMU :
    #Paiwise distance.
    coordinates_diff = ((positions[:, None, :] - positions[None, :, :]))
    coordinates_diff = torch.remainder(coordinates_diff-(SPACE_SIZE/2),SPACE_SIZE)-(SPACE_SIZE/2)
    distances = torch.stack([torch.norm(coordinates_diff[i], dim=1)
                             for i in range(0, len(coordinates_diff))])
    is_greater_than_max = torch.any(distances > MAX_DISTANCE)

    if is_greater_than_max:
      print("At least one distance is greater than the max distance.")

    # force_field calculation
    force_field = force_field_inbox(coordinates_diff, distances, Req=R_EQ,
                                    R0=R_0, Frep=F_REP, Fadh=F_ADH,
                                    coeff_a = COEFF_CARRE, coeff_rep = 0.5)
    
    # displacement computing
    fluctuations = (torch.rand(V0.shape, device=V0.device) - 0.5) * FLUCTUATION_FACTOR
    displacement = MU * force_field * DELTA_T + (V0 + fluctuations) * direction * DELTA_T
    positions += displacement

    # border conditions
    positions = torch.remainder(positions, SPACE_SIZE)

    # Mise à jour de la position et de la direction pour chaque cellule
    for cell, disp in zip(cells, displacement):
        # Mise à jour de la position de la cellule
        cell.position += disp
        cell.position = torch.remainder(cell.position, SPACE_SIZE)
        
        # Mise à jour de la direction de la cellule
        new_direction = autovel(disp.unsqueeze(0), cell.direction.unsqueeze(0), cell.tau, cell.noise, DELTA_T, persistence=cell.persistence)
        cell.direction = new_direction.squeeze(0)
        # Ajouter les informations de la cellule à la liste temporaire
        data_list.append({
            'frame': time,
            'particle': cell.id,
            'pop_tag': cell.pop,  # Inclure le tag de la population ici
            'x': cell.position[0].item(),
            'y': cell.position[1].item(),
            'dir_x': cell.direction[0].item(),
            'dir_y': cell.direction[1].item()
             })

    # plot the result
    if PLOT:
        marker_radius = 1.1
        marker_size = (np.pi) * marker_radius ** 2
        if iteration % PLOT_INTERVAL == 0:
            plot_environment(cells, path_saving = PATH, space_size = SPACE_SIZE, req=R_EQ, iteration = iteration)

    # Après avoir mis à jour toutes les cellules
    direction = torch.stack([cell.direction for cell in cells])
    # Mise à jour du temps
    time += DELTA_T
    # print(time)
    iteration += 1

    # Création du DataFrame à partir de la liste temporaire après la boucle
data_frame = pd.DataFrame(data_list)

In [ ]:
def adjust_positions_xy(group, dt, space_size):
    # Calculer les différences pour obtenir 'dx' et 'dy', en prenant en compte la nature toroïdale
    group['dx'] = group['x'].diff().fillna(0)
    group['dy'] = group['y'].diff().fillna(0)

    # Correction pour la nature toroïdale de l'espace
    group['dx'] = np.where(group['dx'] > space_size / 2, group['dx'] - space_size,
                           np.where(group['dx'] < -space_size / 2, group['dx'] + space_size, group['dx']))
    group['dy'] = np.where(group['dy'] > space_size / 2, group['dy'] - space_size,
                           np.where(group['dy'] < -space_size / 2, group['dy'] + space_size, group['dy']))
    
    # Calculer les nouvelles positions ajustées
    group['adjusted_x'] = group['x'].iloc[0] + group['dx'].cumsum()
    group['adjusted_y'] = group['y'].iloc[0] + group['dy'].cumsum()

    # # Assurer que les valeurs restent dans l'espace toroïdal
    # group['adjusted_x'] %= space_size
    # group['adjusted_y'] %= space_size

    # Calculer la distance parcourue à chaque pas de temps
    group['distance'] = np.sqrt(group['dx']**2 + group['dy']**2)

    # Calculer la vitesse pour chaque pas de temps
    group['vitesse'] = group['distance'] / dt

    # Remplacer les colonnes 'x' et 'y' par les valeurs ajustées
    group['x'], group['y'] = group['adjusted_x'], group['adjusted_y']

    # Supprimer les colonnes temporaires inutiles
    group.drop(['adjusted_x', 'adjusted_y', 'dx', 'dy'], axis=1, inplace=True)

    return group

# Fonction pour calculer l'angle en degrés entre deux vecteurs de direction successifs
def angle_between_directions(row):
    # Extract direction components
    dx1, dy1 = row['dir_x'], row['dir_y']
    dx2, dy2 = row['dir_x_next'], row['dir_y_next']
    
    # Calculate initial and final angles using arctan2 for each direction vector
    angle_initial = np.arctan2(dy1, dx1)
    angle_final = np.arctan2(dy2, dx2)
    
    # Calculate the angle difference
    angle_change = angle_final - angle_initial
    
    if angle_change > np.pi : 
        angle_change = angle_change - 2 * np.pi
    if angle_change < -np.pi:
        angle_change = angle_change + 2 * np.pi

    return angle_change

In [ ]:
# Appliquer la fonction à chaque groupe de particules en utilisant lambda pour inclure DELTA_T comme argument dt
df = data_frame.groupby('particle').apply(lambda group: adjust_positions_xy(group, dt=DELTA_T, space_size=SPACE_SIZE)).reset_index(drop=True)

# Assurez-vous que votre DataFrame est trié par particule et par frame
df.sort_values(by=['particle', 'frame'], inplace=True)

# Décaler les directions pour obtenir le vecteur direction au temps t+1 pour chaque particule
df['dir_x_next'] = df.groupby('particle')['dir_x'].shift(-1)
df['dir_y_next'] = df.groupby('particle')['dir_y'].shift(-1)

# Appliquer la fonction pour calculer l'angle entre les directions successives
df['angle_change'] = df.apply(angle_between_directions, axis=1)

# Optionnel : Supprimer les lignes où la direction suivante est NaN, ce qui se produit pour la dernière observation de chaque particule
df.dropna(subset=['dir_x_next', 'dir_y_next'], inplace=True)

# Calculer la vitesse moyenne pour chaque particule
vitesse_moyenne = df.groupby('particle')['vitesse'].mean()

In [ ]:
# Créer une nouvelle figure
plt.figure(figsize=(10, 6))
# Tracer l'histogramme pour 'angle_change' (normalisé)
plt.hist(df[df['pop_tag']=='Population 1']['angle_change'], bins=100, alpha=0.1, color='blue', label='angle_change pop1', density=True)
plt.hist(df[df['pop_tag']=='Population 2']['angle_change'], bins=100, alpha=0.1, color='red', label='angle_change pop2', density=True)

# Ajouter une légende
plt.legend()
# plt.ylim(0, 0.001)
plt.xlim([-np.pi, np.pi])
# Ajouter des labels et un titre
plt.xlabel('Valeurs')
plt.ylabel('Densité')
plt.title('Histogramme normalisé de rnd et angle change')
# Afficher la figure
plt.show()

In [ ]:
# Créer une figure et deux subplots (axes) sur une grille de 1x2
fig, axs = plt.subplots(1, 2, figsize=(12, 6))  # La figure a 1 ligne et 2 colonnes de graphiques

# Tracer le premier histogramme sur le premier axe (panneau)
axs[0].hist(df['vitesse'], bins = 100)
axs[0].set_title('Histogramme des vitesses instantanées')
axs[0].set_xlabel('Vitesse [um/min]')
axs[0].set_ylabel('Nombre de particules')

# Tracer le second histogramme sur le second axe (panneau)
axs[1].hist(vitesse_moyenne, bins=50)
# axs[1].set_xlim(4, 6)  # Décommentez et ajustez si nécessaire
axs[1].set_title('Histogramme des vitesses moyennes')
axs[1].set_xlabel('Vitesse moyenne [um/min]')
axs[1].set_ylabel('Nombre de particules')

# Ajuster l'espacement entre les subplots pour éviter le chevauchement des titres et des étiquettes
plt.tight_layout()

# Afficher la figure contenant les deux subplots
plt.show()


In [ ]:
fig, ax = plt.subplots()
tp.plot_traj(traj=df)
# Rendre les échelles des axes x et y égales
ax.set_aspect('equal', 'box')
# Ajouter un titre
ax.set_title('Trajectory of Particle 1')
plt.show()

In [ ]:
# Supposons que df_adjusted est votre DataFrame après ajustements avec 'particle', 'x', et 'y'
particle_df = df[df['particle'] == 219]
fig, ax = plt.subplots()
tp.plot_traj(particle_df, mpp=0.637, fps=1, ax=ax)
# Rendre les échelles des axes x et y égales
ax.set_aspect('equal', 'box')
# Ajouter un titre
ax.set_title('Trajectory of Particle 1')
plt.show()

In [ ]:
df_msd = df.copy()
df_msd['frame'] = df_msd.groupby('particle').cumcount() + 1

In [ ]:
def plot_msd(msd, fps, name="MSD of all frames in function of lag time (s)",
             color_plot: str = 'red', save=False, pathway_saving=None,
             alpha=0.05, linewidth=0.01, img_type='jpg'):
    """
    Plot the mean square displacement (MSD) for a specific need.

    Parameters
    ----------
    msd : DataFrame
        DataFrame containing the MSD values.
    fps : float
        Number of frames per second.
    name : str, optional
        Title of the plot. Default is "MSD of all frames in function of lag time (s)".
    save : bool, optional
        Whether to save the plot or not. Default is False.
    pathway_saving : str, optional
        Absolute path to save the plot. Default is None.

    Returns
    -------
    None.

    """
    # Get the number of curves from the number of columns in the MSD DataFrame
    nbr_curves = len(msd.columns)

    # # Set the index
    # msd = msd.set_index("lag time [s]")

    # Create a new figure and axis object
    fig, axis = plt.subplots(figsize=(20, 20))

    # Plot the MSD data on the axis object
    axis.plot(msd, alpha=alpha, linewidth=linewidth, color=color_plot)

    # Set the limits of the x-axis and y-axis
    # axis.set_xlim([1 / fps, 100 / fps])
    # axis.set_ylim(0.01, 10000)

    # Set the x-axis and y-axis to be on a log scale
    axis.set(xscale="log", yscale="log")

    # Set the x-axis label
    axis.set_xlabel("lag time (s)", fontsize=30)

    # Set the x-axis label
    axis.set_ylabel("MSD", fontsize=30)

    # Add a text box to the plot with the number of curves
    textstr = f"nbre curves: {nbr_curves}"
    props = dict(boxstyle="round", facecolor="wheat", alpha=0.5)
    axis.text(0.05, 0.95, textstr, transform=axis.transAxes, fontsize=30,
              verticalalignment="top", bbox=props)

    axis.tick_params(axis='both', which='major', labelsize=20)

    # Set the title of the plot
    fig.suptitle(name, fontsize=40, fontweight="bold", fontstyle='italic', fontname="Arial")

    # Adjust the spacing of the plot
    fig.tight_layout()

    # Save the plot if the "save" parameter is True
    if save:
        fig.savefig(f"{pathway_saving}{name}." + img_type, format=img_type)


IMSD = tp.imsd(df_msd, mpp = 0.637, fps = 0.01)


plot_msd(IMSD, fps=0.01, name="MSD of all frames in function of lag time (s)",
             color_plot = 'red', save=False, pathway_saving=None,
             alpha=1, linewidth=1, img_type='jpg')

In [ ]:
# %% [traj clustering with fit and defining a cutoff]
LAG_TIME_FIT = 5
# plot parameters
IMG_TYPE = 'jpg'
ALPHA = 0.5
LINEWIDTH = 0.1
COLOR_SUP = 'blue'
COLOR_INF = 'red'
color_sup_inf = (COLOR_SUP, COLOR_INF)
# Compute et plot the director factor of the imsd

COEF_INF, COEF_SUP, PART_COEF_INF, PART_COEF_SUP, CUTOFF =\
   lib.traj_clustering_with_fit_cutoff(df, imsd=IMSD, hist=True,
                                        lag_time_fit=LAG_TIME_FIT,
                                        micronperpixel=1,
                                        fps=0.01, binsize=250,
                                        peak_height=50, peak_width=1,
                                        save=False, pathway_fig=None,
                                        name='all the experiment autocorr', img_type="jpg",
                                        plot=True, color_sup_inf=color_sup_inf,
                                        cutoff_default=1.4
                                        )

In [ ]:
# Filtrer le DataFrame pour une particule spécifique
particle_df = df[df['particle'] == 0].iloc[0:10]
particle_df['normdxdy'] = (particle_df['dir_x']**2 + particle_df['dir_y']**2)**0.5
fig, ax = plt.subplots()
# Tracer la trajectoire de la particule
tp.plot_traj(particle_df, mpp=1, fps=15/60, ax = ax)
# Obtenir l'axe actuel
ax = plt.gca()

# Pour chaque point de la trajectoire, ajouter un point représentant la direction
for index, row in particle_df.iterrows():
    # Le point de départ du point (position de la particule)
    
    # Calculer la position du point de direction (en ajoutant un facteur d'échelle aux composantes de direction si nécessaire)
    dir_x = row['x'] + row['dir_x']*0.01   # Facteur d'échelle pour déterminer la position du point de direction
    dir_y = row['y'] + row['dir_y']*0.01  # Facteur d'échelle pour déterminer la position du point de direction
    ax.plot(row['x'], row['y'], 'bo')  # 'ro' crée un point rond rouge
    # Ajouter le point de direction au graphique
    ax.plot(dir_x, dir_y, 'ro')  # 'ro' crée un point rond rouge

# Afficher le graphique
# Assurer les mêmes échelles en x et y
ax.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
# Filtrer le DataFrame pour une particule spécifique
particle_df = df[df['particle'] == 0].iloc[2100:2150]
particle_df['normdxdy'] = (particle_df['dir_x']**2 + particle_df['dir_y']**2)**0.5
fig, ax = plt.subplots()
# Tracer la trajectoire de la particule
tp.plot_traj(particle_df, mpp=1, fps=15/60, ax = ax)
# Obtenir l'axe actuel
ax = plt.gca()

# Pour chaque point de la trajectoire, ajouter un point représentant la direction
for index, row in particle_df.iterrows():
    # Le point de départ du point (position de la particule)
    
    # Calculer la position du point de direction (en ajoutant un facteur d'échelle aux composantes de direction si nécessaire)
    dir_x = row['x'] + row['dir_x']*0.02   # Facteur d'échelle pour déterminer la position du point de direction
    dir_y = row['y'] + row['dir_y']*0.02  # Facteur d'échelle pour déterminer la position du point de direction
    ax.plot(row['x'], row['y'], 'bo')  # 'ro' crée un point rond rouge
    # Ajouter le point de direction au graphique
    ax.plot(dir_x, dir_y, 'ro')  # 'ro' crée un point rond rouge

# Afficher le graphique
# Assurer les mêmes échelles en x et y
ax.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
df

In [ ]:
# Appliquer la fonction à chaque groupe de particules en utilisant lambda pour inclure DELTA_T comme argument dt
df = data_frame.groupby('particle').apply(lambda group: adjust_positions_xy(group, dt=DELTA_T, space_size=SPACE_SIZE)).reset_index(drop=True)

# Assurez-vous que votre DataFrame est trié par particule et par frame
df.sort_values(by=['particle', 'frame'], inplace=True)

# Décaler les directions pour obtenir le vecteur direction au temps t+1 pour chaque particule
df['dir_x_next'] = df.groupby('particle')['dir_x'].shift(-1)
df['dir_y_next'] = df.groupby('particle')['dir_y'].shift(-1)

# Appliquer la fonction pour calculer l'angle entre les directions successives
df['angle_change'] = df.apply(angle_between_directions, axis=1)

# Optionnel : Supprimer les lignes où la direction suivante est NaN, ce qui se produit pour la dernière observation de chaque particule
df.dropna(subset=['dir_x_next', 'dir_y_next'], inplace=True)

# Calculer la vitesse moyenne pour chaque particule
vitesse_moyenne = df.groupby('particle')['vitesse'].mean()

# Tracer l'histogramme des vitesses moyennes
plt.figure(figsize=(10, 6))
plt.hist(vitesse_moyenne, bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.xlabel('Vitesse moyenne [um/min]')
plt.ylabel('Nombre de particules')
plt.title('Histogramme des vitesses moyennes des particules')
plt.grid(True)
plt.show()


In [ ]:
max_step = 10  # Vous pouvez ajuster cette valeur selon vos besoins

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter

# Fonction pour appliquer le filtre de Kalman à une série de mesures
def apply_kalman_filter(measurements, R, Q):
    kf = KalmanFilter(dim_x=1, dim_z=1)
    kf.x = np.array([0.])  # initial state
    kf.F = np.array([[1.]])  # state transition matrix
    kf.H = np.array([[1.]])  # Measurement function
    kf.P = np.array([[1.]])  # covariance matrix
    kf.R = R  # measurement noise
    kf.Q = Q  # process noise
    
    filtered_measurements = []
    for measurement in measurements:
        kf.predict()
        kf.update(np.array([measurement]))
        filtered_measurements.append(kf.x[0])
    
    return filtered_measurements

# Fonction pour sous-échantillonner et calculer les vitesses moyennes pour chaque cellule avec Kalman
def calculate_average_speed_per_cell_with_kalman(df, max_step, delta_t, space_size, R, Q):
    particles = df['particle'].unique()
    steps = range(1, max_step + 1)
    avg_speeds_per_particle = {particle: [] for particle in particles}
    filtered_speeds_per_particle = {particle: [] for particle in particles}
    
    for step in steps:
        sub_sampled_df = df.groupby('particle').apply(lambda group: group.iloc[::step]).reset_index(drop=True)
        sub_sampled_df = sub_sampled_df.groupby('particle').apply(lambda group: adjust_positions_xy(group, dt=delta_t*step, space_size=space_size)).reset_index(drop=True)
        avg_speeds = sub_sampled_df.groupby('particle')['vitesse'].mean()
        
        for particle in particles:
            if particle in avg_speeds:
                avg_speeds_per_particle[particle].append(avg_speeds[particle])
            else:
                avg_speeds_per_particle[particle].append(np.nan)
        
        # Appliquer le filtre de Kalman pour chaque particule
        for particle in particles:
            measurements = np.array(avg_speeds_per_particle[particle])
            filtered_speeds = apply_kalman_filter(measurements, R, Q)
            filtered_speeds_per_particle[particle] = filtered_speeds
    
    return steps, avg_speeds_per_particle, filtered_speeds_per_particle

# Tester différentes valeurs de Q et R
R_values = [2e-1, 1e-1, 5e-2]
Q_values = [1e-1, 1e-5, 1e-10]

# Tracer les résultats pour différentes combinaisons de Q et R
plt.figure(figsize=(15, 10))

for R in R_values:
    for Q in Q_values:
        steps, avg_speeds_per_particle, filtered_speeds_per_particle = calculate_average_speed_per_cell_with_kalman(
            df, max_step, DELTA_T, SPACE_SIZE, R, Q)
        
        # Tracer la vitesse moyenne filtrée pour chaque cellule en fonction de la valeur du pas de temps utilisé
        plt.subplot(len(R_values), len(Q_values), R_values.index(R) * len(Q_values) + Q_values.index(Q) + 1)
        for particle, avg_speeds in filtered_speeds_per_particle.items():
            plt.plot(steps, avg_speeds, marker='o', alpha=0.5, linewidth=0.1, color='blue')
        plt.xlabel('Intervalle de temps (pas)')
        plt.ylabel('Vitesse moyenne filtrée [um/min]')
        plt.title(f'R={R}, Q={Q}')
        plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter

# Fonction pour appliquer le filtre de Kalman à une série de mesures
def apply_kalman_filter(measurements, R, Q):
    kf = KalmanFilter(dim_x=1, dim_z=1)
    kf.x = np.array([0.])  # initial state
    kf.F = np.array([[1.]])  # state transition matrix
    kf.H = np.array([[1.]])  # Measurement function
    kf.P = np.array([[1.]])  # covariance matrix
    kf.R = R  # measurement noise
    kf.Q = Q  # process noise
    
    filtered_measurements = []
    for measurement in measurements:
        kf.predict()
        kf.update(np.array([measurement]))
        filtered_measurements.append(kf.x[0])
    
    return filtered_measurements

# Fonction pour sous-échantillonner et calculer les vitesses moyennes pour chaque cellule avec Kalman
def calculate_average_speed_per_cell_with_kalman(df, max_step, delta_t, space_size, R, Q):
    particles = df['particle'].unique()
    steps = range(1, max_step + 1)
    avg_speeds_per_particle = {particle: [] for particle in particles}
    filtered_speeds_per_particle = {particle: [] for particle in particles}
    avg_speeds_pop1 = []
    avg_speeds_pop2 = []

    for step in steps:
        sub_sampled_df = df.groupby('particle').apply(lambda group: group.iloc[::step]).reset_index(drop=True)
        sub_sampled_df = sub_sampled_df.groupby('particle').apply(lambda group: adjust_positions_xy(group, dt=delta_t*step, space_size=space_size)).reset_index(drop=True)
        avg_speeds = sub_sampled_df.groupby('particle')['vitesse'].mean()
        
        for particle in particles:
            if particle in avg_speeds:
                avg_speeds_per_particle[particle].append(avg_speeds[particle])
            else:
                avg_speeds_per_particle[particle].append(np.nan)
        
        # Appliquer le filtre de Kalman pour chaque particule
        for particle in particles:
            measurements = np.array(avg_speeds_per_particle[particle])
            filtered_speeds = apply_kalman_filter(measurements, R, Q)
            filtered_speeds_per_particle[particle] = filtered_speeds

        # Calculer la vitesse moyenne filtrée pour chaque population
        pop1_particles = sub_sampled_df[sub_sampled_df['pop_tag'] == 'Population 1']['particle'].unique()
        pop2_particles = sub_sampled_df[sub_sampled_df['pop_tag'] == 'Population 2']['particle'].unique()
        
        avg_speed_pop1 = np.nanmean([filtered_speeds_per_particle[p] for p in pop1_particles], axis=0)
        avg_speed_pop2 = np.nanmean([filtered_speeds_per_particle[p] for p in pop2_particles], axis=0)

        avg_speeds_pop1.append(avg_speed_pop1[-1])
        avg_speeds_pop2.append(avg_speed_pop2[-1])
    
    return steps, avg_speeds_per_particle, filtered_speeds_per_particle, avg_speeds_pop1, avg_speeds_pop2

# Tester différentes valeurs de Q et R
R_values = [2e-1, 1e-1, 5e-2]
Q_values = [1e-1, 1e-5, 1e-10]
# Définir les limites des axes
x_limits = (1, max_step)
y_limits = (0, 10)  # Ajustez ces valeurs selon vos données

# Tracer les résultats pour différentes combinaisons de Q et R
plt.figure(figsize=(15, 10))

for R in R_values:
    for Q in Q_values:
        steps, avg_speeds_per_particle, filtered_speeds_per_particle, avg_speeds_pop1, avg_speeds_pop2 = calculate_average_speed_per_cell_with_kalman(
            df, max_step, DELTA_T, SPACE_SIZE, R, Q)
        
        # Tracer la vitesse moyenne filtrée pour chaque cellule en fonction de la valeur du pas de temps utilisé
        plt.subplot(len(R_values), len(Q_values), R_values.index(R) * len(Q_values) + Q_values.index(Q) + 1)
        for particle, avg_speeds in filtered_speeds_per_particle.items():
            plt.plot(steps, avg_speeds, marker='o', alpha=0.5, linewidth=0.1, color='blue')
        
        # Ajouter les courbes de la vitesse moyenne filtrée de chaque population
        plt.plot(steps, avg_speeds_pop1, marker='o', alpha=0.5, linewidth=2, color='green', label='Population 1')
        plt.plot(steps, avg_speeds_pop2, marker='o', alpha=0.5, linewidth=2, color='red', label='Population 2')

        plt.xlabel('Intervalle de temps (pas)')
        plt.ylabel('Vitesse moyenne filtrée [um/min]')
        plt.title(f'R={R}, Q={Q}')
        plt.grid(True)
        plt.legend(loc='upper right')
        plt.xlim(x_limits)
        plt.ylim(y_limits)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter

# Fonction pour appliquer le filtre de Kalman à une série de mesures
def apply_kalman_filter(measurements, R, Q):
    kf = KalmanFilter(dim_x=1, dim_z=1)
    kf.x = np.array([0.])  # initial state
    kf.F = np.array([[1.]])  # state transition matrix
    kf.H = np.array([[1.]])  # Measurement function
    kf.P = np.array([[1.]])  # covariance matrix
    kf.R = R  # measurement noise
    kf.Q = Q  # process noise
    
    filtered_measurements = []
    for measurement in measurements:
        kf.predict()
        kf.update(np.array([measurement]))
        filtered_measurements.append(kf.x[0])
    
    return filtered_measurements

# Fonction pour sous-échantillonner et calculer les vitesses moyennes pour chaque cellule avec Kalman
def calculate_average_speed_per_cell_with_kalman(df, max_step, delta_t, space_size, R, Q):
    particles = df['particle'].unique()
    steps = range(1, max_step + 1)
    avg_speeds_per_particle = {particle: [] for particle in particles}
    filtered_speeds_per_particle = {particle: [] for particle in particles}
    avg_speeds_pop1 = []
    avg_speeds_pop2 = []

    for step in steps:
        sub_sampled_df = df.groupby('particle').apply(lambda group: group.iloc[::step]).reset_index(drop=True)
        sub_sampled_df = sub_sampled_df.groupby('particle').apply(lambda group: adjust_positions_xy(group, dt=delta_t*step, space_size=space_size)).reset_index(drop=True)
        avg_speeds = sub_sampled_df.groupby('particle')['vitesse'].mean()
        
        for particle in particles:
            if particle in avg_speeds:
                avg_speeds_per_particle[particle].append(avg_speeds[particle])
            else:
                avg_speeds_per_particle[particle].append(np.nan)
        
        # Appliquer le filtre de Kalman pour chaque particule
        for particle in particles:
            measurements = np.array(avg_speeds_per_particle[particle])
            filtered_speeds = apply_kalman_filter(measurements, R, Q)
            filtered_speeds_per_particle[particle] = filtered_speeds

        # Calculer la vitesse moyenne filtrée pour chaque population
        pop1_particles = sub_sampled_df[sub_sampled_df['pop_tag'] == 'Population 1']['particle'].unique()
        pop2_particles = sub_sampled_df[sub_sampled_df['pop_tag'] == 'Population 2']['particle'].unique()
        
        avg_speed_pop1 = np.nanmean([filtered_speeds_per_particle[p] for p in pop1_particles], axis=0)
        avg_speed_pop2 = np.nanmean([filtered_speeds_per_particle[p] for p in pop2_particles], axis=0)

        avg_speeds_pop1.append(avg_speed_pop1[-1])
        avg_speeds_pop2.append(avg_speed_pop2[-1])
    
    return steps, avg_speeds_per_particle, filtered_speeds_per_particle, avg_speeds_pop1, avg_speeds_pop2

# Tester différentes valeurs de Q et R
R_values = [3e-1, 1e-1, 1e-1]
Q_values = [1e-1, 1e-5, 1e-10]

# Définir les limites des axes
x_limits = (1, max_step)
y_limits = (0, 10)  # Ajustez ces valeurs selon vos données

# Tracer les résultats pour différentes combinaisons de Q et R
plt.figure(figsize=(15, 10))

for R in R_values:
    for Q in Q_values:
        steps, avg_speeds_per_particle, filtered_speeds_per_particle, avg_speeds_pop1, avg_speeds_pop2 = calculate_average_speed_per_cell_with_kalman(
            df, max_step, DELTA_T, SPACE_SIZE, R, Q)
        
        # Tracer la vitesse moyenne filtrée pour chaque cellule en fonction de la valeur du pas de temps utilisé
        plt.subplot(len(R_values), len(Q_values), R_values.index(R) * len(Q_values) + Q_values.index(Q) + 1)
        for particle, avg_speeds in filtered_speeds_per_particle.items():
            plt.plot(steps, avg_speeds, marker='o', alpha=0.5, linewidth=0.1, color='blue')
        
        # Ajouter les courbes de la vitesse moyenne filtrée de chaque population
        plt.plot(steps, avg_speeds_pop1, marker='o', alpha=0.5, linewidth=2, color='green', label='Population 1')
        plt.plot(steps, avg_speeds_pop2, marker='o', alpha=0.5, linewidth=2, color='red', label='Population 2')

        plt.xlabel('Intervalle de temps (pas)')
        plt.ylabel('Vitesse moyenne filtrée [um/min]')
        plt.title(f'R={R}, Q={Q}')
        plt.grid(True)
        plt.legend(loc='upper right')
        plt.xlim(x_limits)
        plt.ylim(y_limits)

plt.tight_layout()
plt.show()

# Tracer les distributions des vitesses avant et après le filtrage pour quelques particules
particles_to_plot = list(df['particle'].unique())[:5]  # Choisir quelques particules pour tracer

plt.figure(figsize=(15, 10))

for particle in particles_to_plot:
    plt.subplot(3, 2, particles_to_plot.index(particle) + 1)
    original_speeds = np.array(avg_speeds_per_particle[particle])
    filtered_speeds = np.array(filtered_speeds_per_particle[particle])
    
    plt.plot(steps, original_speeds, marker='o', color='red', alpha=0.5, label='Original')
    plt.plot(steps, filtered_speeds, marker='o', color='blue', alpha=0.5, label='Filtered')
    
    plt.xlabel('Intervalle de temps (pas)')
    plt.ylabel('Vitesse [um/min]')
    plt.title(f'Particule {particle}')
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.xlim(x_limits)
    plt.ylim(y_limits)

plt.tight_layout()
plt.show()


In [ ]:
# Fonction pour sous-échantillonner et calculer les vitesses moyennes pour chaque cellule
def calculate_average_speed_per_cell(df, max_step, delta_t, space_size):
    particles = df['particle'].unique()
    steps = range(1, max_step + 1)
    avg_speeds_per_particle = {particle: [] for particle in particles}
    
    for step in steps:
        sub_sampled_df = df.groupby('particle').apply(lambda group: group.iloc[::step]).reset_index(drop=True)
        sub_sampled_df = sub_sampled_df.groupby('particle').apply(lambda group: adjust_positions_xy(group, dt=delta_t*step, space_size=space_size)).reset_index(drop=True)
        avg_speeds = sub_sampled_df.groupby('particle')['vitesse'].mean()
        
        for particle in particles:
            if particle in avg_speeds:
                avg_speeds_per_particle[particle].append(avg_speeds[particle])
            else:
                avg_speeds_per_particle[particle].append(np.nan)
    
    return steps, avg_speeds_per_particle

# Calculer la vitesse moyenne pour chaque cellule et chaque pas de temps
steps, avg_speeds_per_particle = calculate_average_speed_per_cell(df, max_step, DELTA_T, SPACE_SIZE)

# Tracer la vitesse moyenne pour chaque cellule en fonction de la valeur du pas de temps utilisé
plt.figure(figsize=(12, 8))
for particle, avg_speeds in avg_speeds_per_particle.items():
    plt.plot(steps, avg_speeds, marker='o', linewidth=0.1, alpha=0.5, label=f'Cellule {particle}')

plt.xlabel('Intervalle de temps utilisé (nombre de pas)')
plt.ylabel('Vitesse moyenne [um/min]')
plt.title('Vitesse moyenne en fonction de l\'intervalle de temps utilisé pour chaque cellule')
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Déplacer la légende à l'extérieur du graphique
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter

# Fonction pour appliquer le filtre de Kalman à une série de mesures
def apply_kalman_filter(measurements, R=1e-1, Q=1e-5):
    kf = KalmanFilter(dim_x=1, dim_z=1)
    kf.x = np.array([0.])  # initial state
    kf.F = np.array([[1.]])  # state transition matrix
    kf.H = np.array([[1.]])  # Measurement function
    kf.P = np.array([[1.]])  # covariance matrix
    kf.R = R  # measurement noise
    kf.Q = Q  # process noise
    
    filtered_measurements = []
    for measurement in measurements:
        kf.predict()
        kf.update(np.array([measurement]))
        filtered_measurements.append(kf.x[0])
    
    return filtered_measurements

# Fonction pour sous-échantillonner et calculer les vitesses moyennes pour chaque cellule
def calculate_average_speed_per_cell_with_kalman(df, max_step, delta_t, space_size, R=1e-2, Q=1e-5):
    particles = df['particle'].unique()
    steps = range(1, max_step + 1)
    avg_speeds_per_particle = {particle: [] for particle in particles}
    filtered_speeds_per_particle = {particle: [] for particle in particles}
    
    for step in steps:
        sub_sampled_df = df.groupby('particle').apply(lambda group: group.iloc[::step]).reset_index(drop=True)
        sub_sampled_df = sub_sampled_df.groupby('particle').apply(lambda group: adjust_positions_xy(group, dt=delta_t*step, space_size=space_size)).reset_index(drop=True)
        avg_speeds = sub_sampled_df.groupby('particle')['vitesse'].mean()
        
        for particle in particles:
            if particle in avg_speeds:
                avg_speeds_per_particle[particle].append(avg_speeds[particle])
            else:
                avg_speeds_per_particle[particle].append(np.nan)
        
        # Appliquer le filtre de Kalman pour chaque particule
        for particle in particles:
            measurements = np.array(avg_speeds_per_particle[particle])
            filtered_speeds = apply_kalman_filter(measurements, R, Q)
            filtered_speeds_per_particle[particle] = filtered_speeds
    
    return steps, avg_speeds_per_particle, filtered_speeds_per_particle

# Calculer la vitesse moyenne pour chaque cellule et chaque pas de temps
max_step = 10  # Vous pouvez ajuster cette valeur selon vos besoins
steps, avg_speeds_per_particle, filtered_speeds_per_particle = calculate_average_speed_per_cell_with_kalman(
    df, max_step, DELTA_T, SPACE_SIZE, R=1e-2, Q=1e-5)

In [ ]:
# Tracer la vitesse moyenne filtrée pour chaque cellule en fonction de la valeur du pas de temps utilisé
plt.figure(figsize=(12, 8))
for particle, avg_speeds in filtered_speeds_per_particle.items():
    plt.plot(steps, avg_speeds, marker='o', alpha=0.5, linewidth=0.1, color='blue', label=f'Cellule {particle}')

plt.xlabel('Intervalle de temps utilisé (nombre de pas)')
plt.ylabel('Vitesse moyenne filtrée [um/min]')
plt.title('Vitesse moyenne filtrée en fonction de l\'intervalle de temps utilisé pour chaque cellule')
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Déplacer la légende à l'extérieur du graphique
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter

# Fonction pour sous-échantillonner et calculer les vitesses moyennes pour chaque cellule
def calculate_average_speed_per_cell(df, max_step, delta_t, space_size):
    particles = df['particle'].unique()
    steps = range(1, max_step + 1)
    avg_speeds_per_particle = {particle: [] for particle in particles}
    
    for step in steps:
        sub_sampled_df = df.groupby('particle').apply(lambda group: group.iloc[::step]).reset_index(drop=True)
        sub_sampled_df = sub_sampled_df.groupby('particle').apply(lambda group: adjust_positions_xy(group, dt=delta_t*step, space_size=space_size)).reset_index(drop=True)
        avg_speeds = sub_sampled_df.groupby('particle')['vitesse'].mean()
        
        for particle in particles:
            if particle in avg_speeds:
                avg_speeds_per_particle[particle].append(avg_speeds[particle])
            else:
                avg_speeds_per_particle[particle].append(np.nan)
    
    return steps, avg_speeds_per_particle

# Fonction pour appliquer le filtre de Kalman à une série de mesures
def apply_kalman_filter(measurements, R=1e-2, Q=1e-5):
    kf = KalmanFilter(dim_x=1, dim_z=1)
    kf.x = np.array([0.])  # initial state
    kf.F = np.array([[1.]])  # state transition matrix
    kf.H = np.array([[1.]])  # Measurement function
    kf.P = np.array([[1.]])  # covariance matrix
    kf.R = R  # measurement noise
    kf.Q = Q  # process noise
    
    filtered_measurements = []
    for measurement in measurements:
        kf.predict()
        kf.update(np.array([measurement]))
        filtered_measurements.append(kf.x[0])
    
    return filtered_measurements

# Fonction pour sous-échantillonner et calculer les vitesses moyennes pour chaque cellule avec Kalman
def calculate_average_speed_per_cell_with_kalman(df, max_step, delta_t, space_size, R=1e-2, Q=1e-5):
    particles = df['particle'].unique()
    steps = range(1, max_step + 1)
    avg_speeds_per_particle = {particle: [] for particle in particles}
    filtered_speeds_per_particle = {particle: [] for particle in particles}
    
    for step in steps:
        sub_sampled_df = df.groupby('particle').apply(lambda group: group.iloc[::step]).reset_index(drop=True)
        sub_sampled_df = sub_sampled_df.groupby('particle').apply(lambda group: adjust_positions_xy(group, dt=delta_t*step, space_size=space_size)).reset_index(drop=True)
        avg_speeds = sub_sampled_df.groupby('particle')['vitesse'].mean()
        
        for particle in particles:
            if particle in avg_speeds:
                avg_speeds_per_particle[particle].append(avg_speeds[particle])
            else:
                avg_speeds_per_particle[particle].append(np.nan)
        
        # Appliquer le filtre de Kalman pour chaque particule
        for particle in particles:
            measurements = np.array(avg_speeds_per_particle[particle])
            filtered_speeds = apply_kalman_filter(measurements, R, Q)
            filtered_speeds_per_particle[particle] = filtered_speeds
    
    return steps, avg_speeds_per_particle, filtered_speeds_per_particle

# Calculer la vitesse moyenne pour chaque cellule et chaque pas de temps
max_step = 10  # Vous pouvez ajuster cette valeur selon vos besoins
steps, avg_speeds_per_particle = calculate_average_speed_per_cell(df, max_step, DELTA_T, SPACE_SIZE)
steps, avg_speeds_per_particle, filtered_speeds_per_particle = calculate_average_speed_per_cell_with_kalman(
    df, max_step, DELTA_T, SPACE_SIZE, R=1e-1, Q=1e-5)

# Tracer la vitesse moyenne pour chaque cellule en fonction de la valeur du pas de temps utilisé
plt.figure(figsize=(12, 8))

# Tracer les vitesses moyennes non filtrées (en rouge)
for particle, avg_speeds in avg_speeds_per_particle.items():
    plt.plot(steps, avg_speeds, marker='o', linewidth=0.1, alpha=0.5, color='red', label=f'Cellule {particle}' if particle == list(avg_speeds_per_particle.keys())[0] else "")

# Tracer les vitesses moyennes filtrées (en bleu)
for particle, avg_speeds in filtered_speeds_per_particle.items():
    plt.plot(steps, avg_speeds, marker='o', alpha=0.5, linewidth=0.1, color='blue', label=f'Cellule {particle}' if particle == list(filtered_speeds_per_particle.keys())[0] else "")

plt.xlabel('Intervalle de temps utilisé (nombre de pas)')
plt.ylabel('Vitesse moyenne [um/min]')
plt.title('Vitesse moyenne en fonction de l\'intervalle de temps utilisé pour chaque cellule (Non filtrée en rouge, Filtrée en bleu)')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from scipy import stats

# Copier la colonne "frame" en "time" et renuméroter les frames
df['time'] = df['frame']
df['frame'] = df.groupby('particle').cumcount() + 1

# Vérification
print(df.head())


# Fonction pour sous-échantillonner les données
def subsample_data(df, step):
    sub_sampled_df = df.groupby('particle').apply(lambda group: group.iloc[::step]).reset_index(drop=True)
    sub_sampled_df['frame'] = sub_sampled_df.groupby('particle').cumcount() + 1
    return sub_sampled_df

# Fonction pour calculer le MSD
def calculate_msd(df, max_step, delta_t, space_size):
    msd_results = {}
    steps = range(1, max_step + 1)
    
    for step in steps:
        sub_sampled_df = subsample_data(df, step)
        sub_sampled_df = sub_sampled_df.groupby('particle').apply(lambda group: adjust_positions_xy(group, dt=delta_t*step, space_size=space_size)).reset_index(drop=True)
        msd = tp.imsd(sub_sampled_df, mpp=1, fps=1/(delta_t*step))
        msd_results[step] = msd
        print(f'Step {step}: MSD calculated for {len(msd)} particles.')
    
    return msd_results

def calculate_slope_and_intercept(msd, lag_time_fit_short=5, range_long=(0.1, 1)):
    slopes_short = []
    intercepts_short = []
    slopes_long = []
    intercepts_long = []
    
    for col in msd.columns:
        values = msd[col].values  # Convertir en numpy array
        if len(values) >= lag_time_fit_short:
            x_short = np.log(msd.index[:lag_time_fit_short] + 1e-15)
            y_short = np.log(values[:lag_time_fit_short] + 1e-15)
            slope_short, intercept_short, _, _, _ = stats.linregress(x_short, y_short)
            slopes_short.append(slope_short)
            intercepts_short.append(intercept_short)
        # Filtrer les points dans la plage spécifiée pour les temps longs
        mask_long = (msd.index >= range_long[0]) & (msd.index <= range_long[1])
        if np.sum(mask_long) >= lag_time_fit_short:
            x_long = np.log(msd.index[mask_long] + 1e-15)
            y_long = np.log(values[mask_long] + 1e-15)
            slope_long, intercept_long, _, _, _ = stats.linregress(x_long, y_long)
            slopes_long.append(slope_long)
            intercepts_long.append(intercept_long)
    
    return slopes_short, intercepts_short, slopes_long, intercepts_long

# Paramètres
max_step = 10  # Vous pouvez ajuster cette valeur selon vos besoins
delta_t = DELTA_T
space_size = SPACE_SIZE
df_pop1 = df[df['pop_tag']=='Population 1']
df_pop2 = df[df['pop_tag']=='Population 2']

In [ ]:
# Calculer le MSD pour chaque intervalle de temps
msd_results = calculate_msd(df, max_step, delta_t, space_size)

# Couleurs pour les courbes
colors = plt.cm.viridis(np.linspace(0, 1, max_step))